In [1]:
from langchain_google_vertexai import ChatVertexAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

import os
import json
import re

from typing import Set, List

## Загрузка датасета

In [2]:
PATH = "..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

print(len(data))

83755


In [3]:
with open("metrics.json", "r", encoding="utf-8") as file:
    data_metrics = json.load(file)

len(data_metrics)

100

## Объявление LLM

### Т-Pro

In [4]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "../secrets/t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params)

## Utils

In [6]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt

def get_batch(data_metrics,
              examples_str,
              parser = SimpleJsonOutputParser()):
    return [
        {
            "examples": examples_str if examples_str else "Без примеров",
            "format_instructions": parser.get_format_instructions(),
            "problem_title": product["title"],
            "problem_decomposition": json.dumps(product["decomposition"], ensure_ascii=False)        
        } for product in data_metrics
    ]

def get_similarity_results(prompt, 
                           llm,
                           batch, 
                           parser = SimpleJsonOutputParser()):
    similarity_chain = prompt | llm | parser

    return similarity_chain.batch(batch)

def evaluate_predictions(y_true, y_pred):
    y_true = set(y_true)
    y_pred = set(y_pred)
    
    TP = len(y_true & y_pred)  # Совпадающие элементы (правильные предсказания)
    FP = len(y_pred - y_true)  # Ошибочные предсказания (лишние элементы)
    FN = len(y_true - y_pred)  # Пропущенные правильные элементы
    TN = None  # Не учитывается в таких задачах
    
    return TP, FP, FN

def evaluate_batch_predictions(batch_results: List[int]):
    total_TP, total_FP, total_FN = 0, 0, 0
    
    for TP, FP, FN in batch_results:
        total_TP += TP
        total_FP += FP
        total_FN += FN
    
    precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
    recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "TP": total_TP,
        "FP": total_FP,
        "FN": total_FN,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1
    }

def test_prompt(prompt, 
                llm, 
                data_metrics,
                examples_str="",
                get_ans = lambda x: x,
                parser=SimpleJsonOutputParser()):
    batch = get_batch(data_metrics, examples_str, parser)

    results = get_similarity_results(prompt,
                                     llm,
                                     batch,
                                     parser)
    
    y_true_arr = [product["correct_ans"] for product in data_metrics]
    y_pred_arr = [get_ans(res) for res in results]

    scores = [evaluate_predictions(y_true, y_pred) for y_true, y_pred in zip(y_true_arr, y_pred_arr)]

    return [
        {
            "product": product,
            "res": res,
            "score": score
        } for product, res, score in zip(data_metrics, results, scores)
    ]

# Гипотеза один
Просим те ключи, значения которых описывают позицию

### Вариант без CoT

In [13]:
SYSTEM_PROMPT = """
Напиши ключи из характеристики, которая присутствует в названии товара.

Примеры
{examples}

Следуй этим правилам:
- Характеристика должна быть полностью или частично представлена в товарной позиции.
- Включай в ответ ключи, для которых удалось найти соответвутствующи фрагмент в названии товара.

Выдай ответ в виде массива JSON.

**Не пиши больше ничего кроме ответа.**
""".strip()

USER_PROMPT = """
Название товара: {problem_title}
Характеристики: {problem_decomposition}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [8]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" },
        "result": ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"},
        "result": ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": {"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"},
        "result": ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
    }
]

examples_str = "\n".join([f"""
{i+1})Название товара: {product["title"]}
Характеристики: {json.dumps(product['attributes'], ensure_ascii=False)}
Ответ: {json.dumps(product['result'], ensure_ascii=False)} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Название товара: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Характеристики: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Название товара: Сабо женские EVART Ола ЭВА серые (размер 39)
Характеристики: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39",

In [14]:
results = test_prompt(
    prompt,
    llm,
    data_metrics,
    examples_str,
    lambda x: x
)

In [15]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 413,
 'FP': 55,
 'FN': 39,
 'Precision': 0.8824786324786325,
 'Recall': 0.9137168141592921,
 'F1-score': 0.8978260869565218}

### Проверка промпта без примеров

In [11]:
results = test_prompt(
    prompt,
    llm,
    data_metrics,
    "",
    lambda x: x
)

In [12]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 369,
 'FP': 159,
 'FN': 83,
 'Precision': 0.6988636363636364,
 'Recall': 0.8163716814159292,
 'F1-score': 0.7530612244897961}

### Вариант на английском

In [16]:
SYSTEM_PROMPT = """
Write the keys from the characteristic that is present in the product name.

Examples:
{examples}


Pay attention to the strict implementation of these rules:
- The dictionary must consist only of dictionary keys, and the values ​​must be only fragments of the product item.
- Do not write those keys for which there is no match.
- Values ​​should not intersect between other values ​​in your answer.

Give the answer as a JSON array.

**Do not generate any text after the JSON output.**
""".strip()

USER_PROMPT = """
Give an answer for this:
Product title: {problem_title} 
Dictionary: {problem_decomposition}.
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [17]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" },
        "result": ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"},
        "result": ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": {"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"},
        "result": ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
    }
]

examples_str = "\n".join([f"""
{i+1})Product title: {product["title"]}
Dictionary: {json.dumps(product['attributes'], ensure_ascii=False)}
Result: {json.dumps(product['result'], ensure_ascii=False)} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Product title: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Dictionary: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Result: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Product title: Сабо женские EVART Ола ЭВА серые (размер 39)
Dictionary: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "с

In [18]:
results = test_prompt(
    prompt,
    llm,
    data_metrics,
    examples_str,
    lambda x: x
)

In [19]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 423,
 'FP': 91,
 'FN': 29,
 'Precision': 0.8229571984435797,
 'Recall': 0.9358407079646017,
 'F1-score': 0.8757763975155279}

### Вариант с CoT

In [48]:
class SimilarityListResponse(BaseModel):
    reasoning: str = Field(..., description="Ход мыслей перед генерацией списка")
    attributes: List[str] = Field(..., description="Список ключей характеристик")

parser = PydanticOutputParser(pydantic_object=SimilarityListResponse)

SYSTEM_PROMPT = """
Напиши ключи из характеристики, которая присутствует в названии товара.

Примеры
{examples}

Следуй этим правилам:
- Характеристика должна быть полностью или частично представлена в товарной позиции.
- Характеристика должна иметь фрагмент строки для того чтобы понять, что она представлена в товарной позиции.
- Все ключи в ходе мыслей оборачивай в одинарные кавычки - 'текст'

**{format_instructions}**
""".strip()

USER_PROMPT = """
Название товара: {problem_title}
Характеристики: {problem_decomposition}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [49]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" },
        "result": ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"},
        "result": ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": {"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"},
        "result": ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
    }
]

examples_str = "\n".join([f"""
{i+1})Название товара: {product["title"]}
Характеристики: {json.dumps(product['attributes'], ensure_ascii=False)}
Ответ: {json.dumps(product['result'], ensure_ascii=False)} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Название товара: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Характеристики: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Название товара: Сабо женские EVART Ола ЭВА серые (размер 39)
Характеристики: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39",

In [51]:
results = test_prompt(
    prompt,
    llm,
    data_metrics,
    examples_str,
    lambda x: x.attributes,
    parser
)

In [52]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 421,
 'FP': 73,
 'FN': 31,
 'Precision': 0.8522267206477733,
 'Recall': 0.9314159292035398,
 'F1-score': 0.8900634249471459}

In [54]:
for product in results:
    print(product["product"]["title"])
    print(product["res"].reasoning)
    print(f'LLM ans:\t{sorted(product["res"].attributes)}')
    print(f'correct ans:\t{sorted(product["product"]["correct_ans"])}')
    print(product["score"])
    print("---------------")

Тетрадь школьная Комус Класс (№1 School) голубая А5 24 листа в клетку (10 штук в упаковке)
В названии товара мы выделяем ключевые элементы: 'Тетрадь школьная', 'Комус Класс', 'голубая', 'А5', '24 листа в клетку', '10 штук в упаковке'. Соответствующие характеристики следующие: 'Торговая марка' (из 'Комус Класс'), 'Формат листов' (из 'А5'), 'Количество листов' (из '24 листа'), 'Вид линовки' (из 'в клетку'), 'Цвет обложки' (из 'голубая') и 'Количество штук в упаковке' (из '10 штук в упаковке').
LLM ans:	['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов', 'Цвет обложки']
correct ans:	['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов', 'Цвет обложки']
(6, 0, 0)
---------------
Смазка CET CET2828
В названии товара явно указаны 'CET' (в верхнем регистре), что соответствует 'Торговой марке', и 'смазка', что соответствует 'Типу'. Другие характеристики, такие как 'малые закупки', 'страна происхожд

### CoT с порядком действий

In [56]:
class SimilarityListResponse(BaseModel):
    reasoning: str = Field(..., description="""Подумай шаг за шагом, сделай соответствие между фрагментом строки и характеристикой. Рассмотри каждую характеристику по пунктам в формате характеристика-фрагмент""".strip())
    attributes: List[str] = Field(..., description="Ответ на задачу")

parser = PydanticOutputParser(pydantic_object=SimilarityListResponse)

SYSTEM_PROMPT = """
Напиши ключи из характеристик, которые соответствуют товарной позиции.

Примеры:
{examples}

Следуй этим правилам:
- Включать только те характеристики, которые есть в товарной позиции.
- Характеристика может быть частично или полностью представлена в товарной позиции.
- Не включай в ответ ключи, для которых не удалось найти соответствующий фрагмент в описании товара.
- Подтверждение должно быть точным, косвенно не считается.
- Один фрагмент может относиться к нескольким характеристикам.

Затем сформируй массив ключей.
{format_instructions}
Весь ход мыслей напиши в "reasoning". Не пиши ничего кроме JSON.
""".strip()

USER_PROMPT = """
Товарная позиция: {problem_title}
Характеристики: {problem_decomposition}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [57]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" },
        "result": ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"},
        "result": ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": {"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"},
        "result": ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
    }
]

examples_str = "\n".join([f"""
{i+1})Товарная позиция: {product["title"]}
Характеристики: {json.dumps(product['attributes'], ensure_ascii=False)}
Ответ: {json.dumps(product['result'], ensure_ascii=False)} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Характеристики: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39)
Характеристики: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39

In [58]:
results = test_prompt(
    prompt,
    llm,
    data_metrics,
    examples_str,
    lambda x: x.attributes,
    parser
)

In [59]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 428,
 'FP': 74,
 'FN': 24,
 'Precision': 0.852589641434263,
 'Recall': 0.9469026548672567,
 'F1-score': 0.8972746331236897}

# Гипотеза два
Получаем список характеристик, которые отсутствуют в наименовании товарной позиции

## Вариант без CoT

In [35]:
from copy import deepcopy

data_metrics_neg = deepcopy(data_metrics)

for product in data_metrics_neg:
    product["correct_ans"] = list(set(product["decomposition"].keys()) - set(product["correct_ans"]))

In [36]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" },
        "result": ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"},
        "result": ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": {"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"},
        "result": ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
    }
]

examples_str = "\n".join([f"""
{i+1})Товарная позиция: {product["title"]}
Характеристики: {json.dumps(product['attributes'], ensure_ascii=False)}
Ответ: {json.dumps(list(set(product['attributes'].keys()) - set(product['result'])), ensure_ascii=False)} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Характеристики: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Ответ: ["Кол-во единиц продаж в промежуточной упаковке (кратность)", "Длина корпуса карандаша", "Страна происхождения", "Единица продажи", "Количество единиц продаж в транспортной упаковке", "Материал корпуса"]
2)Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39)
Характеристики: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "

In [37]:
SYSTEM_PROMPT = """
Напиши ключи из характеристики, которые отсутствуют в названии товара.

Примеры
{examples}

Следуй этим правилам:
- Характеристика должна быть полностью или частично представлена в товарной позиции.
- Характеристика должна иметь соответвутствующий фрагмент в названии товара.

Выдай ответ в виде массива JSON.

**Не пиши больше ничего кроме ответа.**
""".strip()

USER_PROMPT = """
Название товара: {problem_title}
Характеристики: {problem_decomposition}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [39]:
results = test_prompt(
    prompt,
    llm,
    data_metrics_neg,
    examples_str,
    lambda x: x
)

In [40]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 1101,
 'FP': 32,
 'FN': 243,
 'Precision': 0.971756398940865,
 'Recall': 0.8191964285714286,
 'F1-score': 0.8889786031489705}

## Вариант с CoT

In [41]:
class SimilarityListResponse(BaseModel):
    reasoning: str = Field(..., description="Ход мыслей перед генерацией списка")
    attributes: List[str] = Field(..., description="Список ключей характеристик")

parser = PydanticOutputParser(pydantic_object=SimilarityListResponse)

SYSTEM_PROMPT = """
Напиши ключи из характеристики, которые отсутствуют в названии товара.

Примеры
{examples}

Следуй этим правилам:
- Характеристика должна быть полностью или частично представлена в товарной позиции.
- Характеристика должна иметь соответвутствующий фрагмент в названии товара.

{format_instructions}.

**Не пиши больше ничего кроме ответа.**
""".strip()

USER_PROMPT = """
Название товара: {problem_title}
Характеристики: {problem_decomposition}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [42]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" },
        "result": ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"},
        "result": ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": {"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"},
        "result": ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
    }
]

examples_str = "\n".join([f"""
{i+1})Название товара: {product["title"]}
Характеристики: {json.dumps(product['attributes'], ensure_ascii=False)}
Ответ: {json.dumps(product['result'], ensure_ascii=False)} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Название товара: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Характеристики: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Название товара: Сабо женские EVART Ола ЭВА серые (размер 39)
Характеристики: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39",

In [44]:
results = test_prompt(
    prompt,
    llm,
    data_metrics_neg,
    examples_str,
    lambda x: x.attributes,
    parser
)

In [46]:
evaluate_batch_predictions([product["score"] for product in results])

{'TP': 997,
 'FP': 74,
 'FN': 347,
 'Precision': 0.930905695611578,
 'Recall': 0.7418154761904762,
 'F1-score': 0.825672877846791}

In [47]:
for product in results:
    print(product["product"]["title"])
    print(product["res"].reasoning)
    print(f'LLM ans:\t{sorted(product["res"].attributes)}')
    print(f'correct ans:\t{product["product"]["correct_ans"]}')
    print(product["score"])
    print("---------------")

Тетрадь школьная Комус Класс (№1 School) голубая А5 24 листа в клетку (10 штук в упаковке)
В названии товара упоминаются 'Комус Класс', 'голубая', 'А5', '24 листа в клетку' и '10 штук в упаковке'. Однако не все эти характеристики полностью соответствуют ключам, так как 'голубая' в названии обозначает цвет линовки, а не цвет обложки, как в характеристике 'Цвет обложки'. Также 'Плотность листа бумаги', 'Размер листа', 'Белизна', 'Внутренний блок', 'Отделка обложки', 'Плотность обложки', 'Предмет', и 'Закругленные углы' не представлены в названии.
LLM ans:	['Белизна', 'Внутренний блок', 'Закругленные углы', 'Материал обложки', 'Однотонная обложка', 'Отделка обложки', 'Плотность листа бумаги', 'Плотность обложки', 'Предмет', 'Размер листа', 'Торговая марка']
correct ans:	['Кол-во единиц продаж в промежуточной упаковке (кратность)', 'Страна происхождения', 'Цвет линовки', 'Малые закупки', 'Единица продажи', 'Плотность листа бумаги', 'Предмет', 'Отделка обложки', 'Количество единиц продаж в 